In [17]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

In [18]:
import urllib.request
import pandas as pd
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [19]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [20]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [21]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


In [22]:
from torchtext.legacy import data # torchtext.data 임포트
from konlpy.tag import Mecab

# Mecab을 토크나이저로 사용
tokenizer = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

네이버 영화 리뷰 데이터는 3개의 열로 구성되어져 있으므로 3개의 필드를 정의

In [23]:
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

# print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
# ['영등포구', '청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']

In [24]:
from torchtext.legacy.data import TabularDataset
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [25]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 150000
테스트 샘플의 개수 : 50000


In [26]:
print(vars(train_data[0]))

{'id': '9976970', 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


In [35]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

# print(TEXT.vocab.stoi)
print(dict(list(TEXT.vocab.stoi.items())[:50]))

단어 집합의 크기 : 10002
{'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '는': 4, '영화': 5, '다': 6, '고': 7, '하': 8, '도': 9, '의': 10, '가': 11, '은': 12, '에': 13, '을': 14, '보': 15, '한': 16, '..': 17, '게': 18, ',': 19, '들': 20, '!': 21, '지': 22, '를': 23, '있': 24, '없': 25, '?': 26, '좋': 27, '나': 28, '었': 29, '만': 30, '는데': 31, '너무': 32, '봤': 33, '적': 34, '안': 35, '정말': 36, '로': 37, '음': 38, '으로': 39, '것': 40, '아': 41, '네요': 42, '재밌': 43, '어': 44, '점': 45, '같': 46, '지만': 47, '진짜': 48, '했': 49}


In [42]:
from torchtext.legacy.data import Iterator

batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000


In [43]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text)

tensor([[ 374,   16, 3860,   10,    0,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 383,  206,  323,   10,  338,   16,  338,    3,  149, 1332,   73,  692,
           16,    0,    4, 2677,   34,    3,    7, 1705],
        [  95,  147, 1776,   47,  102,    3,  211,    3,   35,  530,   21,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  79,    9,   25,    7,  396,    9,   25,   47,  417,  210,  208,   55,
           14,   90,    1,    1,    1,    1,    1,    1],
        [ 641,    2,    2,  396,   65,   21,   21,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
